# Notebook to organize Ingestion of XQ100 datasets (v1.0)

In [8]:
# imports
import warnings
from astropy import units as u

from linetools.spectralline import AbsLine
from linetools import utils as ltu
from linetools.isgm.abscomponent import AbsComponent
from linetools.isgm import utils as ltiu
from linetools.analysis import absline as laa


import pyigm
from pyigm.surveys import dlasurvey as dlasvy
from pyigm.abssys.dla import DLASystem

## DLAs

### $N_{\rm HI}$ -- (Sanchez-Ramirez et al. 2016, MNRAS, 456, 4488)

In [2]:
xq100_dla = dlasvy.DLASurvey.load_XQ100(sample='all')
xq100_dla

Loading QSOs file /Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_zpath.fit


<IGMSurvey: nsys=41, type=DLA, ref=XQ-100, nsightlines=100>

In [3]:
xq100_dla._abs_sys[0:5]

[<DLASystem: 00:03:22.9104 -26:03:16.812, 3.39, NHI=21.4, Z/H=0>,
 <DLASystem: 00:06:51.6096 -62:08:03.696, 3.203, NHI=20.9, Z/H=0>,
 <DLASystem: 00:06:51.6096 -62:08:03.696, 3.775, NHI=21, Z/H=0>,
 <DLASystem: 00:34:54.8208 16:39:19.512, 3.755, NHI=20.4, Z/H=0>,
 <DLASystem: 01:13:44.3712 -28:03:17.208, 3.106, NHI=21.2, Z/H=0>]

In [4]:
xq100_dla_coords = xq100_dla.coord
xq100_dla_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(0.84546, -26.05467), (1.71504, -62.13436), (1.71504, -62.13436),
     (8.72842, 16.65542), (18.43488, -28.05478), (21.01571, 0.74244),
     (23.04104, 13.69431), (23.418, 4.01662), (23.418, 4.01662),
     (38.72975, -18.10236), (43.82741, 0.81318), (43.82741, 0.81318),
     (46.84533, -49.76333), (46.84533, -49.76333), (63.81321, -43.96469),
     (66.54304, -22.03814), (82.33671, -35.87614), (116.79647, 27.65093),
     (116.79647, 27.65093), (124.73242, 9.98001), (140.17401, 7.42892),
     (148.75039, -1.50181), (155.16926, 9.38174), (156.2359, 18.31911),
     (164.2724, 19.17857), (164.74326, 12.76526), (167.23113, 12.16482),
     (167.23113, 12.16482), (198.17863, 8.68475), (215.28232, -6.73232),
     (229.4841, 5.18433), (238.22935, 10.09399), (248.3318, 14.19501),
     (260.84667, 22.73278), (339.97333, -5.87194), (356.01333, 3.70722),
     (8.72842, 16.65542), (8.72842, 16.65542), (120.2095, 19.3497),
     (128.79551, 6.848), (238.22935, 10

In [16]:
xq100_dla_zabs = xq100_dla.zabs
xq100_dla_zabs

array([ 3.39  ,  3.203 ,  3.775 ,  3.755 ,  3.106 ,  2.261 ,  3.936 ,
        3.692 ,  3.7725,  3.693 ,  3.9145,  3.2555,  4.4665,  3.591 ,
        3.808 ,  2.9825,  3.684 ,  3.4235,  3.901 ,  3.306 ,  2.238 ,
        4.0245,  2.592 ,  2.298 ,  3.3735,  3.4315,  3.546 ,  3.3965,
        2.66  ,  3.449 ,  2.6885,  3.601 ,  2.882 ,  3.698 ,  4.0805,
        3.22  ,  4.2835,  4.2523,  3.9465,  3.9555,  3.6665])

#### Read Berg table

In [5]:
fil = pyigm.__path__[0]+'/data/DLA/XQ-100/tb_XQ100_metals_to_json.dat'
berg_tbl = Table.read(fil,format='ascii')
berg_tbl[0:5]

qso,ion,wl,zab,zem,ra,dec,vmin,vmax,filename,N,Nerr,flg,flg_analy,lambda,f
str10,str4,int64,float64,float64,str10,str9,int64,int64,str38,float64,float64,int64,int64,float64,float64
J0003-2603,OI,1302,3.39,4.125,00:03:22.8,-26:03:19,-60,70,J0003-2603/tb_J0003-2603_vis_cont.fits,15.1,0.02,3,-1,1302.168,0.04887
J0003-2603,MgI,2852,3.39,4.125,00:03:22.8,-26:03:19,-140,70,J0003-2603/tb_J0003-2603_nir_cont.fits,11.85,0.03,1,0,2852.964,1.81
J0003-2603,MgII,2796,3.39,4.125,00:03:22.8,-26:03:19,-220,200,J0003-2603/tb_J0003-2603_nir_cont.fits,13.91,0.02,2,0,2796.352,0.6123
J0003-2603,MgII,2803,3.39,4.125,00:03:22.8,-26:03:19,-200,170,J0003-2603/tb_J0003-2603_nir_cont.fits,14.04,0.01,2,0,2803.531,0.3054
J0003-2603,CaII,3934,3.39,4.125,00:03:22.8,-26:03:19,-20,50,J0003-2603/tb_J0003-2603_nir_cont.fits,11.81,0.0,3,0,3934.777,0.65


#### Systems

In [6]:
dlas = np.array(['{:s}_z{:0.3f}'.format(row['qso'],row['zab']) for row in berg_tbl])
unique_dlas = np.unique(dlas)
unique_dlas

array(['J0003-2603_z3.390', 'J0006-6208_z3.203', 'J0006-6208_z3.775',
       'J0034+1639_z3.755', 'J0034+1639_z4.252', 'J0034+1639_z4.284',
       'J0113-2803_z3.106', 'J0124+0044_z2.261', 'J0132+1341_z3.936',
       'J0134+0400_z3.692', 'J0134+0400_z3.772', 'J0234-1806_z3.693',
       'J0255+0048_z3.256', 'J0255+0048_z3.914', 'J0307-4945_z3.591',
       'J0307-4945_z4.466', 'J0415-4357_z3.808', 'J0424-2209_z2.982',
       'J0529-3552_z3.684', 'J0747+2739_z3.424', 'J0747+2739_z3.901',
       'J0800+1920_z3.946', 'J0818+0958_z3.306', 'J0835+0650_z3.955',
       'J0920+0725_z2.238', 'J0955-0130_z4.024', 'J1020+0922_z2.592',
       'J1024+1819_z2.298', 'J1057+1910_z3.373', 'J1058+1245_z3.432',
       'J1108+1209_z3.397', 'J1108+1209_z3.546', 'J1312+0841_z2.660',
       'J1421-0643_z3.449', 'J1517+0511_z2.688', 'J1552+1005_z3.601',
       'J1552+1005_z3.667', 'J1633+1411_z2.882', 'J1723+2243_z3.698',
       'J2239-0552_z4.080', 'J2344+0342_z3.220'], 
      dtype='|S17')

#### Loop on DLAs and generate

In [19]:
for cdla in unique_dlas:
    # Rows matching this DLA
    mti = np.where(dlas == cdla)[0]
    # Generate abslines
    abslines = []
    for jj,ii in enumerate(mti):
        row = berg_tbl[ii]
        if jj == 0:
            coord = ltu.radec_to_coord((row['ra'],row['dec']))
            # Match to DLA in XQ-100 survey
            sep = coord.separation(xq100_dla_coords)
            imin = np.argmin(sep)
            mt = np.where((sep < 5*u.arcsec) & (np.abs(row['zab']-xq100_dla_zabs) < 1e-2))[0]
            if len(mt) != 1:
                xdb.set_trace()
            if sep[mt[0]] > 1*u.arcsec:
                warnings.warn("DLA {} has a large coordinate offset = {}".format(cdla,sep[mt].to('arcsec')))
            dla = xq100_dla._abs_sys[mt[0]]
            # Reset DLA
            dla.zabs = row['zab']
            dla._components= []
            print("Working on DLA {}".format(dla))
        if row['ion'] == 'TiII':
            print('Skipping TiII')
            continue
        #
        trans = '{:s} {:d}'.format(row['ion'],row['wl'])
        #print('DLA={:s}, trans={:s}'.format(cdla,trans))
        aline = AbsLine(trans)
        # Coord, zem, etc. attrib
        aline.attrib['coord'] = dla.coord
        aline.attrib['z'] = row['zab']
        aline.attrib['logN'] = row['N']
        aline.attrib['sig_logN'] = row['Nerr']
        aline.attrib['flag_N'] = row['flg']
        laa.linear_clm(aline.attrib)
        #xdb.set_trace()
        # Analysis
        aline.analy['vlim'] = [row['vmin'], row['vmax']]*u.km/u.s
        aline.analy['do_analysis'] = row['flg_analy']+1
        aline.analy['spec'] = row['filename']
        # Checks
        np.testing.assert_allclose(aline.wrest.value, row['lambda'], rtol=1e-4)
        try:
            np.testing.assert_allclose(aline.data['f'], row['f'], rtol=1e-4)
        except AssertionError:
            pass
            #print("f-values do not closely match for trans={:s}".format(trans))
            #print("f_lt={:g}, f_Berg={:g}".format(aline.data['f'], row['f']))
        # Add
        abslines.append(aline)
    # Generate components and add them in
    comps = ltiu.build_components_from_abslines(abslines, chk_vel=False)
    for comp in comps:
        comp.synthesize_colm(overwrite=True)
        dla.add_component(comp)
    break

Working on DLA <DLASystem: 00:03:22.9104 -26:03:16.812, 3.39, NHI=21.4, Z/H=0>
Skipping TiII


/Users/xavier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: UserWarning: DLA J0003-2603_z3.390 has a large coordinate offset = [ 2.65977323] arcsec


In [23]:
dla.fill_ionN()
dla._ionN

Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN
,,,1 / cm,,km / s,km / s,,,
int64,int64,int64,float64,float64,float64,float64,int64,float64,float64
8,1,0,0.0,3.39,-60.0,70.0,0,0.0,0.0
12,1,0,0.0,3.39,-140.0,70.0,1,11.85,0.03
12,2,0,0.0,3.39,-220.0,200.0,2,14.04,0.01
14,2,0,0.0,3.39,-200.0,70.0,2,14.37,0.01
15,2,0,0.0,3.39,-20.0,50.0,3,13.73,0.0
20,2,0,0.0,3.39,-20.0,50.0,3,11.81,0.0
24,2,0,0.0,3.39,-40.0,75.0,1,13.07,0.04
25,2,0,0.0,3.39,-90.0,60.0,3,12.39,0.06
